In [ ]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

import zmq

In [ ]:
# set up ZMQ to publish tweets
context = zmq.Context()
socket = context.socket(zmq.PUB)
socket.bind('tcp://0.0.0.0:5555')

In [ ]:
# create Twitter Stream Listener 
class LiveStreamListener(StreamListener):
    
    def on_data(self, data):
        # to confirm streamer is listening
        #print(data)
        
        # send data over zmq
        socket.send_json(data)
        
        return True

    def on_error(self, status):
        print(status)

In [ ]:
# read in the app credentials from external file
creds = {}
credentials_file = 'credentials-vm1-tw-app'

with open(credentials_file, 'r') as f:
    for line in f.readlines():
        line = line.split(':')
        creds[line[0]] = line[1].strip()

In [ ]:
# instantiate listener and set up authentication
listener = LiveStreamListener()
auth = OAuthHandler(creds['consumer_key'], creds['consumer_secret'])
auth.set_access_token(creds['access_token'], creds['access_token_secret'])

In [ ]:
# create stream object
stream = Stream(auth, listener)

while True:
    # filter for just the data you want from the stream
    stream.filter(track = ['verizon', 'humanability'])